### Test 1: executing excel file
this requires to be execute either in macOS or windows

In [ ]:
import xlwings as xw

In [ ]:
wb = xw.Book("model.xlsx")
wb.app.calculate()
result = wb.sheets["Sheet1"]["B2"].value


### Test 2: trying to replicate the calculus that excel does

In [1]:
from openpyxl import load_workbook
from openpyxl.formula.translate import Translator

# from subprocess import Popen
import formulas

from pathlib import Path

In [ ]:
# path to the Excel file
file_path = Path("../raw_data/original_tools/Blue_Carbon_Cost_ep_v3_v1.xlsm")
output_path = Path("../raw_data/original_tools/Blue_Carbon_Cost_ep_v3_v1_output.xlsm")

In [2]:
# Load the workbook
wb = load_workbook(file_path.as_posix(), keep_vba=True, keep_links=True)

In [ ]:
xl_model = formulas.ExcelModel().loads(file_path.as_posix()).finish()
xl_model.calculate()
xl_model.write(dirpath=output_path.as_posix())

in the excel the main sheet is called "Dashboard", from here data needs to be inputted in the next cells:
1. 


In [ ]:
print("Input data")
for i in range(6, 14):
    print(
        wb["Dashboard"].cell(row=i, column=2).value,
        ": ",
        wb["Dashboard"].cell(row=i, column=5).value,
    )

ranges_ = {"Restoration": [15, 22], "Protection": [23, 26]}
print(wb["Dashboard"].cell(row=11, column=5).value)
range_to_print = ranges_[wb["Dashboard"].cell(row=11, column=5).value]

for i in range(*range_to_print):
    print(i)
    print(
        wb["Dashboard"].cell(row=i, column=2).value,
        ": ",
        wb["Dashboard"].cell(row=i, column=5).value,
    )


print("Output data")
print("___________________________")
for i in range(6, 10):
    print(
        wb["Dashboard"].cell(row=i, column=7).value,
        ": ",
        wb["Dashboard"].cell(row=i, column=9).value,
    )

In [ ]:
# Select the active worksheet
sheet = workbook.active

# Read data from a specific cell
print("Original value in A1:", sheet["A1"].value)

# Modify data in a specific cell
sheet["A1"] = "Modified Value"

# Save the changes to the workbook
workbook.save("example_modified.xlsx")


print("Workbook modified and saved as 'example_modified.xlsx'")


### Test 3: Execute excel in office360 remotely using python

In [ ]:
import msal
import requests
import json


In [ ]:
class GraphExcelClient:
    def __init__(self, client_id, client_secret, tenant_id, file_path):
        self.client_id = client_id
        self.client_secret = client_secret
        self.tenant_id = tenant_id
        self.file_path = file_path
        self.graph_root = "https://graph.microsoft.com/v1.0"
        self.token = self._authenticate()
        self.headers = {"Authorization": f"Bearer {self.token}", "Content-Type": "application/json"}
        self.item_id = self._get_file_id()

    def _authenticate(self):
        authority = f"https://login.microsoftonline.com/{self.tenant_id}"
        scopes = ["https://graph.microsoft.com/.default"]
        app = msal.ConfidentialClientApplication(
            self.client_id, authority=authority, client_credential=self.client_secret
        )
        result = app.acquire_token_silent(scopes, account=None)
        if not result:
            result = app.acquire_token_for_client(scopes=scopes)
        if "access_token" not in result:
            raise Exception("Authentication failed: ", result.get("error_description"))
        return result["access_token"]

    def _get_file_id(self):
        resp = requests.get(
            f"{self.graph_root}/me/drive/root:{self.file_path}", headers=self.headers
        )
        resp.raise_for_status()
        return resp.json()["id"]

    def set_values(self, worksheet, address, values):
        url = f"{self.graph_root}/me/drive/items/{self.item_id}/workbook/worksheets('{worksheet}')/range(address='{address}')"
        payload = {"values": values}
        resp = requests.patch(url, headers=self.headers, json=payload)
        resp.raise_for_status()
        print(f"✅ Updated values in {worksheet}!{address}")

    def calculate(self, calc_type="Full"):
        url = f"{self.graph_root}/me/drive/items/{self.item_id}/workbook/application/calculate"
        payload = {"calculationType": calc_type}
        resp = requests.post(url, headers=self.headers, json=payload)
        resp.raise_for_status()
        print(f"✅ Workbook recalculated using '{calc_type}'")

    def get_values(self, worksheet, address):
        url = f"{self.graph_root}/me/drive/items/{self.item_id}/workbook/worksheets('{worksheet}')/range(address='{address}')"
        resp = requests.get(url, headers=self.headers)
        resp.raise_for_status()
        values = resp.json()["values"]
        print(f"Retrieved values from {worksheet}!{address}: {values}")
        return values


In [ ]:
class ExcelMappingClient:
    def __init__(self, excel_client, input_mapping, output_mapping):
        """
        excel_client: The instance of GraphExcelClient to interact with the workbook.
        input_mapping: Dict mapping input parameters to their ranges (cells or tables).
        output_mapping: Dict mapping output parameters to their ranges (cells or tables).
        """
        self.excel_client = excel_client
        self.input_mapping = input_mapping
        self.output_mapping = output_mapping

    def _handle_api_error(self, response, action="API request"):
        """
        General API error handler to check response status and raise exceptions.
        """
        if not response.ok:
            error_message = response.json().get("error", {}).get("message", "Unknown error")
            raise Exception(f"Error in {action}: {error_message}")
        return response.json()

    def _apply_mapping(self, mapping, is_input=True):
        """
        Applies the mapping to the workbook using set_values for input or get_values for output.
        """
        sheet_wise = {}

        for param, config in mapping.items():
            sheet = config.get("sheet", "Sheet1")
            range_or_table = config.get("range", None)  # Can be a range or table name

            # Handle missing range or table
            if not range_or_table:
                raise KeyError(f"Missing 'range' for parameter '{param}' in mapping config.")

            # Validate that values exist for inputs
            if is_input and "values" not in config:
                raise KeyError(f"Missing 'values' for parameter '{param}' in input mapping.")

            # Handle table or range input/output
            if is_input:
                values = config.get("values", None)
                if range_or_table and values:
                    if sheet not in sheet_wise:
                        sheet_wise[sheet] = {}
                    sheet_wise[sheet][range_or_table] = values
            else:
                if range_or_table:
                    if sheet not in sheet_wise:
                        sheet_wise[sheet] = []
                    sheet_wise[sheet].append(range_or_table)

        return sheet_wise

    def apply_inputs(self, input_data):
        """
        Applies input data to the workbook.
        input_data: {'parameter': value} where 'parameter' is the mapped name.
        """
        for param, value in input_data.items():
            if param not in self.input_mapping:
                raise KeyError(f"Input parameter '{param}' not in mapping.")
            config = self.input_mapping[param]
            config["values"] = [[value]]

            # Ensure the values are in the correct format (list of lists)
            if not isinstance(config["values"], list) or not isinstance(config["values"][0], list):
                raise ValueError(f"Invalid value format for '{param}'. Expected a list of lists.")

            # Attempt to set values in the Excel sheet and handle any API errors
            try:
                response = self.excel_client.set_values(
                    config["sheet"], config["range"], config["values"]
                )
                self._handle_api_error(response, f"Setting input value for '{param}'")
            except Exception as e:
                print(f"❌ Error applying input for '{param}': {e}")

        print("✅ Inputs applied successfully.")

    def apply_outputs(self):
        """
        Retrieves output values from the workbook.
        """
        output_values = {}
        for param, _ in self.output_mapping.items():
            config = self.output_mapping[param]
            try:
                # Attempt to get the value for each output parameter
                response = self.excel_client.get_values(config["sheet"], config["range"])
                self._handle_api_error(response, f"Retrieving output value for '{param}'")
                output_values[param] = response
            except Exception as e:
                print(f"❌ Error retrieving output for '{param}': {e}")
                output_values[param] = None  # Fallback in case of error

        return output_values

    def handle_complex_ranges(self, mapping, is_input=True):
        """
        Handles complex ranges (like tables, named ranges) by checking for table structure and applying.
        """
        for param, config in mapping.items():
            range_or_table = config.get("range", None)
            sheet = config.get("sheet", "Sheet1")

            if isinstance(range_or_table, str) and range_or_table.startswith("table:"):
                # Handle table input or output (e.g., setting rows for table)
                table_name = range_or_table.split(":")[1]
                if is_input:
                    # Apply table rows as values (simply a sample - you'd implement logic for rows)
                    try:
                        response = self.excel_client.set_values(
                            sheet, f"table:{table_name}", config["values"]
                        )
                        self._handle_api_error(response, f"Setting table values for '{param}'")
                    except Exception as e:
                        print(f"❌ Error applying table input for '{param}': {e}")
                else:
                    try:
                        response = self.excel_client.get_values(sheet, f"table:{table_name}")
                        self._handle_api_error(response, f"Retrieving table output for '{param}'")
                        return response
                    except Exception as e:
                        print(f"❌ Error retrieving table output for '{param}': {e}")
            elif isinstance(range_or_table, str) and range_or_table.startswith("named:"):
                # Handle named ranges (similarly for named ranges)
                named_range = range_or_table.split(":")[1]
                if is_input:
                    try:
                        response = self.excel_client.set_values(
                            sheet, f"named:{named_range}", config["values"]
                        )
                        self._handle_api_error(
                            response, f"Setting named range values for '{param}'"
                        )
                    except Exception as e:
                        print(f"❌ Error applying named range input for '{param}': {e}")
                else:
                    try:
                        response = self.excel_client.get_values(sheet, f"named:{named_range}")
                        self._handle_api_error(
                            response, f"Retrieving named range output for '{param}'"
                        )
                        return response
                    except Exception as e:
                        print(f"❌ Error retrieving named range output for '{param}': {e}")
            else:
                # Handle regular cell ranges
                try:
                    if is_input:
                        response = self.excel_client.set_values(
                            sheet, range_or_table, config["values"]
                        )
                        self._handle_api_error(response, f"Setting cell values for '{param}'")
                    else:
                        response = self.excel_client.get_values(sheet, range_or_table)
                        self._handle_api_error(response, f"Retrieving cell value for '{param}'")
                        return response
                except Exception as e:
                    print(f"❌ Error applying or retrieving value for '{param}': {e}")

    def compute(self, input_data):
        """
        Combines input and output operations.
        First, apply inputs and then retrieve outputs.
        """
        # Apply inputs
        self.apply_inputs(input_data)
        # Calculate the workbook
        self.excel_client.calculate()
        # Apply outputs (fetch results)
        output_results = self.apply_outputs()
        return output_results


In [ ]:
# Configuration
# inputs and outputs (including complex ranges)
input_mapping = {
    "price": {"sheet": "Sheet1", "range": "B2", "values": [[200]]},
    "quantity": {"sheet": "Sheet1", "range": "C2", "values": [[5]]},
    "discount": {"sheet": "Sheet1", "range": "D2", "values": [[0.1]]},
    "order_table": {
        "sheet": "Sheet1",
        "range": "table:Orders",
        "values": [["Product A", 150, 2]],
    },  # Example of table input
}

output_mapping = {
    "total": {"sheet": "Sheet1", "range": "D3"},
    "final_price": {"sheet": "Sheet1", "range": "E3"},
    "order_table_results": {"sheet": "Sheet1", "range": "table:Orders"},
}

CLIENT_ID = "your-client-id"
CLIENT_SECRET = "your-client-secret"
TENANT_ID = "your-tenant-id"
FILE_PATH = "/Documents/model.xlsx"

# Initialize client
excel = GraphExcelClient(CLIENT_ID, CLIENT_SECRET, TENANT_ID, FILE_PATH)

# # Set inputs
# excel.set_values("Sheet1", "A2:B2", [[200, 3]])  # e.g., Price = 200, Quantity = 3

# # Recalculate
# excel.calculate()

# # Get output
# result = excel.get_values("Sheet1", "C2")  # e.g., Total = Price * Quantity


In [ ]:
# Sample input data
input_data = {
    "price": 200,
    "quantity": 5,
    "discount": 0.1,
}

# Initialize the client (GraphExcelClient)
excel_client = GraphExcelClient(CLIENT_ID, CLIENT_SECRET, TENANT_ID, "/Documents/model.xlsx")
app = ExcelMappingClient(excel_client, input_mapping, output_mapping)

# Apply inputs and retrieve outputs
output_values = app.compute(input_data)

# Print output results
print("Output Values:")
for param, value in output_values.items():
    print(f"{param}: {value}")
